# 01 — Overview (Outbound migration pressure over time)

**Dataset (source file):** `raw_data/hdx_hapi_refugees_afg.csv`  
**Website section:** `js/sections/overview.js`

This notebook:
1) reads the raw file,
2) applies the same filters as the website,
3) saves a **cleaned** dataset to `dataset/cleaned/refugees_outflow_clean.csv`,
4) shows which rows were removed (and why, via the filter mask),
5) produces derived outputs: `dataset/derived/outflow_by_year.csv` and `.json`.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

RAW_DIR = Path("raw_data")        # raw inputs live here
DATASET_DIR = Path("dataset")     # website reads from here
CLEAN_DIR = DATASET_DIR / "cleaned"
DERIVED_DIR = DATASET_DIR / "derived"

CLEAN_DIR.mkdir(parents=True, exist_ok=True)
DERIVED_DIR.mkdir(parents=True, exist_ok=True)

def to_number(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s.astype(str).str.replace(",", "").str.strip(), errors="coerce")

def parse_year(val) -> float:
    if pd.isna(val):
        return np.nan
    s = str(val).strip()
    if len(s) >= 4 and s[:4].isdigit():
        return float(s[:4])
    for sep in ("/", "-"):
        parts = s.split(sep)
        if len(parts) == 3 and parts[-1].isdigit():
            return float(parts[-1])
    return np.nan

def show_filter_report(df_before: pd.DataFrame, mask: pd.Series, title: str, sample_n: int = 10):
    kept = df_before[mask].copy()
    dropped = df_before[~mask].copy()
    print(f"=== {title} ===")
    print("rows_before:", len(df_before))
    print("rows_kept:", len(kept))
    print("rows_dropped:", len(dropped))
    if len(dropped):
        print("\nSample dropped rows:")
        display(dropped.head(sample_n))
    return kept, dropped


In [ ]:
raw_path = RAW_DIR / "hdx_hapi_refugees_afg.csv"
df = pd.read_csv(raw_path)
df.head()


In [ ]:
required = {"origin_location_code","asylum_location_code","reference_period_start","population"}
missing = sorted(required - set(df.columns))
assert not missing, f"Missing columns: {missing}"

df2 = df.copy()
df2["year"] = df2["reference_period_start"].apply(parse_year)
df2["population_num"] = to_number(df2["population"])

mask = (df2["origin_location_code"].astype(str).str.strip() == "AFG") &        (df2["asylum_location_code"].astype(str).str.strip() != "AFG") &        (df2["year"].notna()) &        (df2["population_num"].notna()) &        (df2["population_num"] > 0)

if "population_group" in df2.columns:
    g = df2["population_group"].astype(str).str.strip().str.upper()
    mask = mask & ((g == "") | (g.isin(["REF","ASY"])))

kept, dropped = show_filter_report(df2, mask, "Refugees/outflow filter (AFG origin, dest != AFG, valid year, pop>0, group REF/ASY)")


In [ ]:
# Save cleaned (only fields used for the outflow story)
clean = kept.copy()
clean["year"] = clean["year"].astype(int)
clean["population_num"] = clean["population_num"].fillna(0).astype(float)

clean_out = CLEAN_DIR / "refugees_outflow_clean.csv"
clean.to_csv(clean_out, index=False)
clean_out


In [ ]:
# Aggregate outflow by year (same as overview.js)
outflow = (clean.groupby("year", as_index=False)["population_num"].sum()
             .sort_values("year"))
outflow.rename(columns={"population_num":"outflow_ref_asy"}, inplace=True)
outflow.head(), outflow.tail()


In [ ]:
plt.figure()
plt.plot(outflow["year"], outflow["outflow_ref_asy"])
plt.title("Outbound migration pressure (REF+ASY) over time")
plt.xlabel("Year")
plt.ylabel("People (count)")
plt.show()


In [ ]:
out_csv = DERIVED_DIR / "outflow_by_year.csv"
out_json = DERIVED_DIR / "outflow_by_year.json"
outflow.to_csv(out_csv, index=False)
outflow.set_index("year")["outflow_ref_asy"].to_json(out_json, indent=2)
(out_csv, out_json)


In [ ]:
# Optional: save dropped rows for auditing
dropped_out = CLEAN_DIR / "refugees_outflow_dropped.csv"
dropped.to_csv(dropped_out, index=False)
dropped_out
